# ETL data into CCBISDW

ETL会有各种方法：
* Tools
  * Export/Import (For streit data migration, no transformation included)
  * SSIS/Data Fabric for cloud
  * Informatica
  * Talent
  * Pantoho
  * Nifi
  * etc,
* Programming
  * Python
  * T-SQL (Inside SQL Server amount databaseses)
  * Java
  * C#
  * R
  * SAS
  * etc. <br>

我们会根据具体的情况综合考虑一下因素：
* Hadware/Software const
* Development
* Maintenance
* Performance

开发过程的code会多一点，因为会有一些实验性质的code，完成后进入production之前我们会做code cleaning，所以code就会比较简洁，而且会schedule job自动执行。
另外也会加一些注释，为日后维护提供方便。一般code也会check in到source control，比如最常用的就是git。

这里我们选择Python programming做ETL

## Generate dataframe from CSV

In [33]:
## From SQL to DataFrame Pandas
import pandas as pd
import pyodbc
import os
import pandas as pd
#!pip install sqlalchemy
import sqlalchemy

from sqlalchemy import create_engine, MetaData, Table, select
from six.moves import urllib

os.chdir(r"C:\Courses\DA-Project\Data")
ccbis_files=os.listdir()
createVar = locals()
for i in ccbis_files:
    if i.endswith("csv"):
        createVar[i.split('.')[0]] = pd.read_csv(i)

## Create a database connection engine

In [9]:
params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=localhost;DATABASE=CCBISDW;UID=sa;PWD=SQLServer2019")
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params) 
engine.connect() 

## Load DimAgent Table

In [10]:
DimAgent.to_sql(name='DimAgent',con=engine, index=False, if_exists='append')

## Load DimServerifyType table

In [17]:
DimSeverifyType.to_sql(name='DimSeverifyType',con=engine, index=False, if_exists='append')

## Load DimServiceType Table

In [35]:
DimServiceType.to_sql(name='DimServiceType',con=engine, index=False, if_exists='append')

## Load DimHandleType Table

In [28]:
DimHandleType.to_sql(name='DimHandleType',con=engine, index=False, if_exists='append')

## Load DimProduct Table

In [15]:
DimProductGroup.columns = ['ProductGroup_Key','ProductGroup']
DimPro = pd.merge(DimProduct,DimProductGroup, on='ProductGroup_Key')
DimPro = DimPro[['Product_Key','Name','ProductGroup']]
DimPro.to_sql(name='DimProduct',con=engine, index=False, if_exists='append')

## Load Dimcustomer Table

In [27]:
#DimProductGroup.columns = ['ProductGroup_Key','ProductGroup']
DimCust = pd.merge(DimCustomer,DimGeography, on='GeographyKey')
DimCust = DimCust.loc[:, DimCust.columns != 'GeographyKey']
DimCust.to_sql(name='DimCustomer',con=engine, index=False, if_exists='append')

## Load FactCDR

In [36]:
CDR.to_sql(name='FactCDR',con=engine, index=False, if_exists='append')